In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import sys
print(sys.executable)

C:\Users\jenas\OneDrive\Desktop\House_prediction\.venv\Scripts\python.exe


In [3]:
DATA_DIR = Path(r"C:\Users\jenas\OneDrive\Desktop\House_prediction\data\raw")

train_df = pd.read_csv(DATA_DIR / r"train.csv")
test_df  = pd.read_csv(DATA_DIR / r"test.csv")

train_df.shape, test_df.shape

((1460, 81), (1459, 80))

# Problem Definition

- Task: Supervised Regression
- Target Variable: SalePrice
- Input: 80 features in X (after dropping target)
- Evaluation Metric: RMSE (later)

In [4]:
target = "SalePrice"
X = train_df.drop(columns=[target])
y = train_df[target]

X.shape, y.shape

((1460, 80), (1460,))